In [12]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, Flatten
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle
import numpy as np

In [2]:
# Load the reviews and labels from the file
with open('train_reviews.pkl', 'rb') as f:
    train_reviews, train_labels = pickle.load(f)

with open('test_reviews.pkl', 'rb') as f:
    test_reviews, test_labels = pickle.load(f)

In [7]:
# Load the tokenized and padded sequences
x_train = np.load('x_train.npy')
x_test = np.load('x_test.npy')

# Load the one-hot encoded labels
y_train = np.load('y_train.npy')
y_test = np.load('y_test.npy')

# Load the tokenizer
with open('tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

max_words = 10000
max_review_length = 500

In [8]:
# Define the CNN model architecture
embedding_vector_length = 32
model = Sequential()
model.add(Embedding(max_words, embedding_vector_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=5, batch_size=64)

# Save the model
model.save('CNN_model.keras')

C:\Users\DELL\anaconda3\Lib\site-packages\keras\src\layers\core\embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 12s 23ms/step - accuracy: 0.6454 - loss: 0.5773
Epoch 2/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - accuracy: 0.9322 - loss: 0.1825
Epoch 3/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - accuracy: 0.9583 - loss: 0.1215
Epoch 4/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - accuracy: 0.9794 - loss: 0.0702
Epoch 5/5
391/391 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.9919 - loss: 0.0384


In [9]:
# Load the model
model = load_model('CNN_model.keras')

In [10]:
# Predict the test set
y_pred = model.predict(x_test)

# Convert predictions to class labels
y_pred_labels = [1 if pred[1] > pred[0] else 0 for pred in y_pred]

# Calculate and print accuracy, precision, recall, and F1 score
accuracy = accuracy_score(test_labels, y_pred_labels)
precision = precision_score(test_labels, y_pred_labels)
recall = recall_score(test_labels, y_pred_labels)
f1 = f1_score(test_labels, y_pred_labels)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")

782/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step
Accuracy: 0.87
Precision: 0.88
Recall: 0.86
F1 Score: 0.87


In [16]:
# Take user input for the review
input_review = input("Enter a review: ")

# Preprocessing function
def preprocess_review(review, tokenizer, maxlen):
    # Convert the review to a sequence of integers
    sequences = tokenizer.texts_to_sequences([review])
    # Pad the sequences to ensure uniform input length
    padded_sequences = pad_sequences(sequences, maxlen=maxlen)
    return padded_sequences

# Preprocess the input review
preprocessed_review = preprocess_review(input_review, tokenizer, max_review_length)

# Predict the sentiment of the input review
y_pred = model.predict(preprocessed_review)

# Convert prediction to class label (1 for positive, 0 for negative)
y_pred_label = 1 if y_pred[0][1] > y_pred[0][0] else 0

# Print the predicted sentiment
sentiment = "positive" if y_pred_label == 1 else "negative"
print(f"The predicted sentiment for the review is: {sentiment}")

Enter a review: I'm not a big fan of musicals, although this technically might not qualify as a musical. But I thought I would give it a chance as I love war movies. It was mediocre at best.<br /><br />Hudson seems totally out of kilter in this role. It just didn't work for me. Julie Andrews probably played her part as best as she could, but I just find it hard to buy her as a conniving, deceptive spy. Sorry, I know that is classic stereotyping on my part. But I have to say I think this is Julie at her most beautiful and feminine looking. I always thought of her as more matronly, but then surely that's a result of her roles in Sound of Music and Mary Poppins. No doubt they were desperately trying to get her out of that typecasting in this role. She was quite beguiling in appearance here, but I still didn't buy her as a spy.<br /><br />I couldn't keep my focus through the whole movie and found myself tuning in and out - and having conversations with those in my room (which I usually nev